In [34]:
#EXERCISE 1 : IMPLEMENTATION OF NAIVE BAYES ALGORITHM
import pandas as pd
import numpy as np

import kagglehub
  # Download latest version
path = kagglehub.dataset_download("renanmav/imdb-movie-review-dataset")
print("Path to dataset files:",path)

Using Colab cache for faster access to the 'imdb-movie-review-dataset' dataset.
Path to dataset files: /kaggle/input/imdb-movie-review-dataset


In [35]:
import os

os.listdir(path)
movie_review_path = os.path.join(path,"movie_data.csv")
df =pd.read_csv(movie_review_path)

In [36]:
reviews = df['review'].values
#print(reviews.lower())

In [37]:
print(reviews)

['This movie is just crap. Even though the directors claim to be part of that oi-culture, it\'s still a very, very bad directorial debut. The topic itself is very interesting and I accept the bad acting due to the fact, that they are all amateurs and never acted before, but the worst thing about this film are the dialogs and very unexperienced and naive directing. There\'s no timing at all in that movie. I felt like the directors were so exited to do that movie (it\'s their first feature), that they actually never really asked themselves, what story they wanna tell. I met Ben (one of the directors) on several occasions and he\'s a nice and thoughtful guy, but that doesn\'t make him a director. I think, that "American History X" is full of clichÃ©s, but somehow manages to transport a story. "Oi!Warning" is full of clichÃ©s, doesn\'t tell anything new or provocative and (-that\'s the sad thing about this movie) it\'s far from any Oi!-Reality.<br /><br />If you wanna see weird but great G

In [38]:
reviews.size

49969

In [39]:
reviews_lower = [text.lower() for text in reviews]


In [40]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
#remove non-alpha calues
import re
def clean_text(text, remove_stopwords=True, use_stemming=False):
  text = text.lower()
  text=re.sub(r'[^a-zA-Z\s]','',text)
  words = text.split()
  if remove_stopwords:
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]

  if use_stemming:
    stemmer = PorterStemmer()
    words =[stemmer.stem(w) for w in words]
  else:
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]

  return ' '.join(words)
sample_text = "Hello! This is a SAMPLE email with numbers 123 and special #tags."
print("Original:", sample_text)
print("Cleaned:", clean_text(sample_text))

Original: Hello! This is a SAMPLE email with numbers 123 and special #tags.
Cleaned: hello sample email number special tag


In [42]:
#CountVectorizer Example
from sklearn.feature_extraction.text import CountVectorizer

# Sample documents
documents = [
    "I love machine learning",
    "Machine learning is amazing",
    "I hate spam emails",
    "Spam emails are annoying"
]

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Fit and transform
X_counts = count_vectorizer.fit_transform(documents)

print("Vocabulary:", count_vectorizer.get_feature_names_out())
print("\nDocument-Term Matrix shape:", X_counts.shape)
print("\nMatrix (dense representation):")
print(X_counts.toarray())

Vocabulary: ['amazing' 'annoying' 'are' 'emails' 'hate' 'is' 'learning' 'love'
 'machine' 'spam']

Document-Term Matrix shape: (4, 10)

Matrix (dense representation):
[[0 0 0 0 0 0 1 1 1 0]
 [1 0 0 0 0 1 1 0 1 0]
 [0 0 0 1 1 0 0 0 0 1]
 [0 1 1 1 0 0 0 0 0 1]]


In [47]:
df['sentiment'].head(50)

,sentiment
0,0
1,1
2,0
3,0
4,0
5,0
6,0
7,1
8,0
9,1


In [49]:
#1.a
df['cleaned_review'] = df["review"].apply(lambda x: clean_text(x, remove_stopwords=True, use_stemming=True))

print("Original review example:")
print(df.loc[0, 'review'])
print("\nCleaned review example:")
print(df.loc[0, 'cleaned_review'])
print(f"\nBinary labels: {df['sentiment'].unique()}")

Original review example:
This movie is just crap. Even though the directors claim to be part of that oi-culture, it's still a very, very bad directorial debut. The topic itself is very interesting and I accept the bad acting due to the fact, that they are all amateurs and never acted before, but the worst thing about this film are the dialogs and very unexperienced and naive directing. There's no timing at all in that movie. I felt like the directors were so exited to do that movie (it's their first feature), that they actually never really asked themselves, what story they wanna tell. I met Ben (one of the directors) on several occasions and he's a nice and thoughtful guy, but that doesn't make him a director. I think, that "American History X" is full of clichÃ©s, but somehow manages to transport a story. "Oi!Warning" is full of clichÃ©s, doesn't tell anything new or provocative and (-that's the sad thing about this movie) it's far from any Oi!-Reality.<br /><br />If you wanna see we

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re

In [50]:

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_review'],
    df['sentiment'],
    test_size=0.2,
    random_state=42,
    stratify=df['sentiment']
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")
print(f"\nTraining class distribution:\n{y_train.value_counts(normalize=True)}")

Training set: 39975 samples
Testing set: 9994 samples

Training class distribution:
sentiment
0    0.500038
1    0.499962
Name: proportion, dtype: float64


In [51]:
vectorizer = CountVectorizer(max_features=3000)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [52]:
# Initialize and train the model
nb_classifier = MultinomialNB(alpha=1.0)  # alpha=1 for Laplace smoothing
nb_classifier.fit(X_train_vec, y_train)
print("Model trained successfully!")

Model trained successfully!


In [53]:
y_pred = nb_classifier.predict(X_test_vec)
y_pred_proba = nb_classifier.predict_proba(X_test_vec)

In [55]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8358014808885331


In [56]:
print("Classification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.84      0.84      4998
           1       0.84      0.83      0.84      4996

    accuracy                           0.84      9994
   macro avg       0.84      0.84      0.84      9994
weighted avg       0.84      0.84      0.84      9994



In [57]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[4195  803]
 [ 838 4158]]


In [60]:
from sklearn.metrics import roc_auc_score

In [67]:
print(y_pred_proba)

[[9.99997337e-01 2.66293101e-06]
 [9.25547929e-01 7.44520710e-02]
 [9.99795700e-01 2.04300444e-04]
 ...
 [2.54081943e-04 9.99745918e-01]
 [3.00843784e-03 9.96991562e-01]
 [1.01045259e-01 8.98954741e-01]]


In [68]:
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])
print("ROC-AUC Score:", roc_auc)


ROC-AUC Score: 0.9053769826585558
